In [17]:
import numpy as np
import nltk
from nltk import bigrams
import string
from collections import defaultdict

In [18]:
toy_corpus = ["This is the house that Jack built", "This is the malt", 
              "That lay in the house that Jack built", "This is the rat",
             "That ate the malt", "That lay in the house that Jack built",
             "This is the cat", "That killed the rat", "That ate the malt", 
             "That lay in the house that Jack built"]

In [26]:
model = defaultdict(lambda : defaultdict(lambda : 0))

In [27]:
vocab = {"<start>", "<end>"}
for sents in toy_corpus:
    sents = sents.lower()
    sents = f"<start> {sents} <end>"
    for w1, w2 in bigrams(sents.split()):
        model[w1][w2] += 1
        vocab.add(w1)
        vocab.add(w2)

In [28]:
model

defaultdict(<function __main__.<lambda>()>,
            {'<start>': defaultdict(<function __main__.<lambda>.<locals>.<lambda>()>,
                         {'this': 4, 'that': 6}),
             'this': defaultdict(<function __main__.<lambda>.<locals>.<lambda>()>,
                         {'is': 4}),
             'is': defaultdict(<function __main__.<lambda>.<locals>.<lambda>()>,
                         {'the': 4}),
             'the': defaultdict(<function __main__.<lambda>.<locals>.<lambda>()>,
                         {'house': 4, 'malt': 3, 'rat': 2, 'cat': 1}),
             'house': defaultdict(<function __main__.<lambda>.<locals>.<lambda>()>,
                         {'that': 4}),
             'that': defaultdict(<function __main__.<lambda>.<locals>.<lambda>()>,
                         {'jack': 4, 'lay': 3, 'ate': 2, 'killed': 1}),
             'jack': defaultdict(<function __main__.<lambda>.<locals>.<lambda>()>,
                         {'built': 4}),
             'built': defau

In [30]:
vocab_size = len(vocab)
print(f"Vocab Size : {vocab_size}")

Vocab Size : 16


# Add One Smoothning

<br/>
<br/>
As explained in the previous notebook that total_count calculates the occurence of a word in the model. Now to apply the equation of Add-1 Smoothning:
$$
\begin{equation}
p(w_{i}|w^{i-1}_{i-n+1}) = \frac{c(w^{i}_{i-n+1}) + 1}{c(w^{i-1}_{i-n+1}) + V}
\end{equation}
$$

I have written this line of code :
```
model[w1][w2] = (model[w1][w2] + 1) / (total_count + vocab_size)
```

Now how is this line code of representing the above equation, well as you can see that
for the `w2`, I am taking those words out of the vocabulary.
<br/>
<br/>
So in my model let's say the current state is as follows:
```
{
that : {jack : 4, lay : 3, ate : 2, killed : 1},
this : {is : 4}
}
```

So there is no combination for `that is` or `this lay` in the model. 
<br/>
<br/>
But after the equation for the above line of code my model will then have the following representation:
```
{
that:{jack: 0.2380, lay : 0.2, ate : 0.1578, killed : 0.1111, is : 0.0588},
this : {is : 0.2380, jack : 0.0588, .....}
}
```

So this incorporates all the combinations that can happen hence the model perplexity won't be infinite

In [32]:
for w1 in model:
    total_count = float(sum(model[w1].values()))
    for w2 in vocab:
        model[w1][w2] = (model[w1][w2] + 1) / (total_count + vocab_size) 
        
model

defaultdict(<function __main__.<lambda>()>,
            {'<start>': defaultdict(<function __main__.<lambda>.<locals>.<lambda>()>,
                         {'this': 0.19230769230769232,
                          'that': 0.2692307692307692,
                          'house': 0.038461538461538464,
                          '<end>': 0.038461538461538464,
                          'is': 0.038461538461538464,
                          'malt': 0.038461538461538464,
                          'rat': 0.038461538461538464,
                          'ate': 0.038461538461538464,
                          'in': 0.038461538461538464,
                          'built': 0.038461538461538464,
                          'killed': 0.038461538461538464,
                          '<start>': 0.038461538461538464,
                          'cat': 0.038461538461538464,
                          'the': 0.038461538461538464,
                          'jack': 0.038461538461538464,
                          'lay': 

In [35]:
start_token = "this"
sents = [start_token]
complete = False

while not complete:
    
    sorted_words = [(k, v) for k, v in sorted(model[start_token].items(), 
                                             key = lambda x: x[1], 
                                              reverse = True)]
    
    if len(sorted_words) == 0:
        break
        
        
    sents.append(sorted_words[0][0])
    
    start_token = sorted_words[0][0]
    
    if start_token == "<end>":
        complete = True
        

final_sents = ""

for tokens in sents:
    if tokens not in ["<start>", "<end>"]:
        final_sents += tokens + " "
print(final_sents)

this is the house that jack built 
